In [1]:
import pandas as pd

# Dados de treino
X_treino = pd.read_csv('../Dados/dados_tratados/X_treino.csv')
y_treino = pd.read_csv('../Dados/dados_tratados/y_treino.csv').values.ravel()

In [2]:
# Parâmetros
SEMENTE_ALEATORIA = 9 # Random state
num_max_atributos = len(X_treino.columns) - 1

X_treino = X_treino.values

In [3]:
from sklearn.linear_model import LogisticRegression # algoritmo de aprendizado de máquina
from sklearn.preprocessing import StandardScaler, MinMaxScaler # normalização
from sklearn.feature_selection import RFE # seleção de atributos
from sklearn.decomposition import PCA # redução de dimensionalidade
from sklearn.pipeline import make_pipeline

def cria_instancia_modelo(trial):
    """Cria uma instância do modelo.

    Args:
      trial: objeto tipo Trial do optuna.

    Returns:
      Uma instância do modelo desejado.

    """
    parametros = {
        "penalty": "elasticnet",
        "l1_ratio": trial.suggest_float("razão_regularização", 0, 1), # 0 -> l2, 1 -> l1
        "solver": "saga", # única capaz de resolver elasticnet
        "max_iter": 5000,
        "class_weight": "balanced", # Dados desbalanceados
        "n_jobs": -1,
        "random_state": SEMENTE_ALEATORIA,
    }

    normalizacao = trial.suggest_categorical("normalizacao", ["standard", "minmax"])
    usar_pca = trial.suggest_categorical("PCA", [True, False])
    usar_rfe = trial.suggest_categorical("RFE", [True, False])

    num_atributos = trial.suggest_int("num_atributos", 2, num_max_atributos) # para RFE
    num_dimensoes = trial.suggest_int("num_dimensoes", 2, num_atributos) # para PCA
        
    
    steps = []

    if normalizacao == "standard":
        steps.append(("normalizador", StandardScaler()))
    else:
        steps.append(("normalizador", MinMaxScaler()))

    if usar_rfe:
        steps.append(("rfe", RFE(LogisticRegression(**parametros), n_features_to_select=num_atributos)))

    if usar_pca:
        steps.append(("pca", PCA(n_components=num_dimensoes, random_state=SEMENTE_ALEATORIA)))

    steps.append(("modelo", LogisticRegression(**parametros)))

    modelo = make_pipeline(*[s[1] for s in steps])

    return modelo

In [4]:
from sklearn.model_selection import cross_val_score
from sklearn.metrics import fbeta_score, make_scorer
import numpy as np

f2_score = make_scorer(fbeta_score, beta=2)

def funcao_objetivo(trial, X, y, num_folds):
    """Função objetivo do optuna.

    Faz validação cruzada estratificada com métrica F2.

    """
    modelo = cria_instancia_modelo(trial)

    metricas = cross_val_score(
        modelo,
        X,
        y,
        scoring=f2_score,
        cv=num_folds,
    )

    return np.mean(metricas)

In [5]:
from optuna import create_study

NOME_DO_ESTUDO = "Logistic_Reg_optuna"

objeto_de_estudo = create_study(
    direction="maximize",
    study_name=NOME_DO_ESTUDO,
    storage=f"sqlite:///{NOME_DO_ESTUDO}.db",
    load_if_exists=True,
)

[I 2025-10-30 21:38:12,420] Using an existing study with name 'Logistic_Reg_optuna' instead of creating a new one.


In [7]:
# Parâmetros
NUM_FOLDS = 10 # Validação cruzada
NUM_TENTATIVAS = 200 # Quantidade de trials

def funcao_objetivo_parcial(trial):
    return funcao_objetivo(trial, X_treino, y_treino, NUM_FOLDS)

# Faz a otimização
#objeto_de_estudo.optimize(funcao_objetivo_parcial, n_trials=NUM_TENTATIVAS)

In [8]:
df_analysis = objeto_de_estudo.trials_dataframe()

df_analysis

,number,value,datetime_start,datetime_complete,duration,params_PCA,params_RFE,params_normalizacao,params_num_atributos,params_num_dimensoes,params_razão_regularização,state
0,0,0.838777,2025-10-30 20:51:14.969684,2025-10-30 20:51:15.749928,0 days 00:00:00.780244,False,False,standard,7,6,0.156461,COMPLETE
1,1,0.784501,2025-10-30 20:51:16.057093,2025-10-30 20:51:16.574865,0 days 00:00:00.517772,True,False,minmax,4,2,0.760751,COMPLETE
2,2,0.855771,2025-10-30 20:51:16.638876,2025-10-30 20:51:17.186701,0 days 00:00:00.547825,True,True,minmax,8,3,0.718492,COMPLETE
3,3,0.852464,2025-10-30 20:51:17.300743,2025-10-30 20:51:18.176032,0 days 00:00:00.875289,False,False,minmax,5,2,0.856811,COMPLETE
4,4,0.838777,2025-10-30 20:51:18.233851,2025-10-30 20:51:19.024704,0 days 00:00:00.790853,False,False,standard,3,2,0.076628,COMPLETE
...,...,...,...,...,...,...,...,...,...,...,...,...
205,205,0.862489,2025-10-30 21:38:15.555808,2025-10-30 21:38:16.063806,0 days 00:00:00.507998,True,False,minmax,5,5,0.466669,COMPLETE
206,206,0.862489,2025-10-30 21:38:16.112196,2025-10-30 21:38:16.553885,0 days 00:00:00.441689,True,False,minmax,5,5,0.379953,COMPLETE
207,207,0.862489,2025-10-30 21:38:16.609771,2025-10-30 21:38:17.114984,0 days 00:00:00.505213,True,False,minmax,5,5,0.402569,COMPLETE
208,208,0.862489,2025-10-30 21:38:17.148751,2025-10-30 21:38:17.618935,0 days 00:00:00.470184,True,False,minmax,5,5,0.425720,COMPLETE


In [9]:
melhor_trial = objeto_de_estudo.best_trial

print(f"Número do melhor trial: {melhor_trial.number}")
print(f"Parâmetros do melhor trial: {melhor_trial.params}")

Número do melhor trial: 108
Parâmetros do melhor trial: {'razão_regularização': 0.46499984523274707, 'normalizacao': 'minmax', 'PCA': True, 'RFE': False, 'num_atributos': 6, 'num_dimensoes': 5}


Referência optuna: https://medium.com/@walter_sperat/using-optuna-with-sklearn-the-right-way-part-1-6b4ad0ab2451